In [1]:
import pandas as pd
import numpy as np
from time import time
from os.path import isfile

# convert the DataFrame in .csv into matrix
def csv_to_matrix(df_read, shape=[10000, 1000]):
    A = np.zeros(shape)
    
    id_string = df_read['Id'].str.split('_')
    row_ids = id_string.str[0].str.extract('([0-9]+)', expand=False).astype('int32')
    col_ids = id_string.str[1].str.extract('([0-9]+)', expand=False).astype('int32')
    
    A[row_ids-1, col_ids-1] = df_read['Prediction']
    return A

# Write the results from matrix to .csv and matches with the samplesubmission
def matrix_to_csv(df_read, A):
    id_string = df_read['Id'].str.split('_')
    row_ids = id_string.str[0].str.extract('([0-9]+)', expand=False).astype('int32')
    col_ids = id_string.str[1].str.extract('([0-9]+)', expand=False).astype('int32')
    
    df_read['Prediction'] = A[row_ids-1, col_ids-1]
    
    return df_read


In [2]:
# Define the size of the training data
user_N = 100000
item_N = 1000


# Load the training data from matrix (if exist) or .csv

train_data_matrix_file = './train_data_matrix.npy'

if isfile('./train_data_matrix.npy'):
    A = np.load(train_data_matrix_file)
else:
    df_train = pd.read_csv('./cil-collab-filtering-2018/data_train.csv')
    
    # Rearrange the data in .csv into a sparse matrix
    A = csv_to_matrix(df_train, [user_N, item_N])
    np.save('./train_data_matrix.npy', A)

In [3]:
# Compute average for nonzero colomn elemens and fill into A
for j in range(A.shape[1]):
    ave = A[np.nonzero(A[:, j]), j].mean()
    A[np.where(A[:,j]==0),j] = ave

In [4]:
tic = time()
df_submit = pd.read_csv('./cil-collab-filtering-2018/sampleSubmission.csv')
submit_csv = matrix_to_csv(df_submit, A)
submit_csv.to_csv('./cil-collab-filtering-2018/submission.csv', index=False)
toc = time()
print('Total number of entries in the submission file: %d' % np.shape(submit_csv)[0])
print('Submission writing time: %f s' % (toc-tic))

Total entries in the submission file: 1176952
Submission writing time: 8.463201 s
